# Zarr vs Zarr-HDF read time comparison
Compute the maximum water level during Hurricane Ike on a 9 million node triangular mesh storm surge model.  

In [ ]:
import xarray as xr
from dask.distributed import Client, progress, performance_report
from dask_kubernetes import KubeCluster
import fsspec

from zarr.storage import FileChunkStore

### Start a dask cluster to crunch the data

In [ ]:
cluster = KubeCluster()

In [ ]:
cluster.scale(30);

In [ ]:
cluster

In [ ]:
client = Client(cluster)

In [ ]:
client

### Read the data using the Zarr

In [ ]:
%%time
ds = xr.open_zarr(fsspec.get_mapper('s3://pangeo-data-uswest2/esip/adcirc/adcirc_01d', anon=False, requester_pays=True))

Compute the max water level at each grid cell (reads all the data):

In [ ]:
%%time
with performance_report(filename="dask-zarr-report.html"):
    max_var1 = ds['zeta'].max(dim='time').compute()

### Read the data using the Zarr-HDF5 Connection

The content of the netCDF4 file with the storm surge model output was translated to Zarr metadata but the data in file's variables was not copied over. The generated Zarr metadata has an additional information on where all the chunks are located in the file. Access to these chunks is enabled via a _chunk store_, a different store just for the Zarr array chunks. The _chunk store_, in this case, reads chunk data directly from the netCDF4 file.

In [ ]:
%%time
store = fsspec.get_mapper('s3://hdf5-zarr/adcirc_01d.nc.chunkstore', anon=True)

ncfile = fsspec.open('s3://pangeo-data-uswest2/esip/adcirc/adcirc_01d.nc', mode='rb', 
                     anon=False, requester_pays=True, default_fill_cache=False)

chunk_store = FileChunkStore(store, chunk_source=ncfile.open())

ds = xr.open_zarr(store, consolidated=True, chunk_store=chunk_store)

In [ ]:
%%time
with performance_report(filename="dask-hdf5-report.html"):
    max_var2 = ds['zeta'].max(dim='time').compute()

Check a value to see if the two methods match

In [ ]:
max_var1[1000].values

In [ ]:
max_var2[1000].values